In [17]:
import pyautogui as pag
import numpy as np
import mss, cv2, win32gui, win32ui, win32con, pdb, time, mouse
import matplotlib.pyplot as plt
import copy
import multiprocess as mp

In [370]:
pos = pag.position()
print(pos)

Point(x=250, y=835)


In [74]:
def check_num(color):
    global color_list
    color_list = np.array([[255, 246, 222], # 0 
                           [255, 254, 255], # 2
                           [255, 236, 170], # 4
                           [255, 229, 143], # 8
                           [255, 216,  87], # 16
                           [241, 183,   0], # 32
                           [204, 140,   1], # 64
                           [ 60, 240, 141], # 128
                           [ 32, 232, 109], # 256z
                           [ 26, 222,  77], # 512
                           [121, 225, 255], # 1024
                           [ 13, 201, 255], # 2048 
                           [  1, 151, 255]]) # 4096
    color = color.flatten()
    loc = np.argmin(np.sum((color_list - color)**2, axis=1))
    return loc.item()

def get_tile_array(tile_xpos, tile_ypos):
    tile_array = np.zeros((4, 4))
    for ii in range(4):
        for jj in range(4):
            with mss.mss() as sct:
                color = np.array(sct.grab(
                    {'left':tile_xpos[ii], 'top':tile_ypos[jj], 
                     'width':1, 'height':1}))[:, :, :3]
                pass
            # print(color)
            tile_array[jj, ii] = check_num(color)
    return tile_array

def after_moving(tile_array, direction='up'):
    if direction == 'up':
        tile_array0 = copy.deepcopy(tile_array)
    if direction == 'down':
        tile_array0 = copy.deepcopy(tile_array[::-1])
    if direction == 'right':
        tile_array0 = copy.deepcopy(tile_array.T[::-1])
    if direction == 'left':
        tile_array0 = copy.deepcopy(tile_array.T)

    templet = np.zeros((4, 4))
    for i in range(4): # summing
        part = tile_array0[:, i]
        part0 = part[part > 0]
        for j in range(len(part0)-1):
            if part0[j] == part0[j+1]:
                part0[j] += 1
                part0[j+1] = 0
        part1 = part0[part0 > 0]
        templet[:len(part1), i] = part1

    if direction == 'down':
        templet = templet[::-1]
    if direction == 'right':
        templet = templet[::-1].T
    if direction == 'left':
        templet = templet.T
    return templet

def evaluation(after_tile_array, table):
    # print(table)
    dum = after_tile_array
    dum = np.where(after_tile_array==0, 2, after_tile_array)
    return np.sum(table*(2**dum))

def find_direct(tile_array, depth=5):
    if np.sum(2**tile_array) > 2**10: return 'up', -1
    dir_order = ['left', 'up', 'right', 'down']
    if (depth % 2) == 1:
        score = np.zeros(len(dir_order))
        for ii, try_dir in enumerate(dir_order):
            after = after_moving(tile_array, direction=try_dir)
            if (after == tile_array).all(): 
                score[ii] = -1
            elif depth == 1:
                score[ii] = evaluation(after, table)
            else:
                score[ii] = find_direct(after, depth=depth-1)
        return dir_order[np.argmax(score)], np.max(score)    
    else : #depth // 2 == 0:
        empty = np.where(tile_array == 0)
        dir_score = []
        for ii in range(len(empty[0])):
            for jj in range(2):
                tile_array[empty[0][ii], empty[1][ii]] = jj
                direct_sub, score_sub = find_direct(tile_array, depth-1)
                dir_score.append(score_sub)
        return np.mean(dir_score)

def drag(direction, delay=1e-5):
    delta = 30
    x, y = 220, 779
    if direction=='up': mouse.drag(x, y, x, y-delta)
    if direction=='down': mouse.drag(x, y, x, y+delta)    
    if direction=='right': mouse.drag(x, y, x+delta, y)
    if direction=='left': mouse.drag(x, y, x-delta, y)


In [75]:
challenge_mode_pos = [220, 719]
practice_mode_pos = [220, 779]
tile_xpos = np.arange(4)*100 + 50
tile_ypos = np.arange(4, dtype=int)*100 + 550

global table
table0 = np.arange(4*4)[::-1].reshape(4, 4)
table = 2.**(table0)
# table = table0**2
table[1] = table[1][::-1]
table[3] = table[3][::-1]
# table[0, 0] *= 2
# table[0, 0] = table[0, 0]*1e4
# table = table * 10**np.arange(4)[::-1, None]

# table = np.array([[10,8,7,6.5],
#  [.5,.7,1,3],
#  [-.5,-1.5,-1.8,-2],
#  [-3.8,-3.7,-3.5,-3]])

# aa = 2**np.arange(4)[::-1]
# table = aa*aa[:, None]

In [ ]:
mode = 'challenge'
mode = 'practice'
start_pos = challenge_mode_pos if mode =='challenge' else practice_mode_pos
pag.click(x = start_pos[0], y = start_pos[1])
time.sleep(3)
err = 0
while True:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir, score = find_direct(tile_array, depth = 3)
    err = err + 1 if score == -1 else 0
    if err == 10: break
    drag(final_dir)
    # if (final_dir == 'right') and \
    #     ((after_moving(tile_array, 'right'))[0, 0] == 0):
    #     drag('left')
    time.sleep(0.1)

In [38]:
err

0

In [39]:
score

np.float64(499311.0)

In [53]:
tile_array = get_tile_array(tile_xpos, tile_ypos)
tile_array

array([[6., 4., 1., 4.],
       [2., 3., 5., 3.],
       [2., 4., 1., 2.],
       [1., 1., 2., 1.]])

In [54]:
np.sum(tile_array)

np.float64(42.0)

In [ ]:
while True:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir, score = find_direct(tile_array, 0)
    print("\r", final_dir, score, end="")
    # time.sleep(0)

In [71]:
# tile_array = np.array([[3, 2, 2, 4], 
#                        [1, 2, 2, 0], 
#                        [0, 0, 0, 0], 
#                        [1, 1, 0, 0]])
tile_array = get_tile_array(tile_xpos, tile_ypos)
final_dir, score = find_direct(tile_array)   
final_dir, score

('left', np.float64(-1.0))

In [73]:
get_tile_array(tile_xpos, tile_ypos)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [67]:
tile_array == after_moving(tile_array, 'left')

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])